# E09 Movielens 영화 추천

-  MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작
- MovieLens 1M Dataset
- 별점 데이터를 Implicit 데이터로 간주하고 테스트 ( 별점을 시청횟수로 해석해서 생각)
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외



## 1. 데이터 준비와 전처리

In [81]:
import pandas as pd
PATH = '/content/drive/MyDrive/Github/AIFFEL/E09'

In [82]:
rating_file_path= PATH + '/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(3)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [83]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [84]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)

In [85]:
ratings.head(3)

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [131]:
# 영화 제목을 보기 위해 메타 데이터 읽어오기
movie_file_path= PATH + '/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [132]:
#movie_id로 inner join
data = pd.merge(movies, ratings, left_on='movie_id', right_on='movie_id', how='inner')
data.head()

,movie_id,title,genre,user_id,count,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [133]:
# # 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'title', 'count']
data = data[using_cols]
data.head(10)

,user_id,title,count
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [134]:
#검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.
data['title'] = data['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,title,count
0,1,toy story (1995),5
1,6,toy story (1995),4
2,8,toy story (1995),4
3,9,toy story (1995),5
4,10,toy story (1995),5
5,18,toy story (1995),4
6,19,toy story (1995),5
7,21,toy story (1995),3
8,23,toy story (1995),4
9,26,toy story (1995),3


In [135]:
#첫번째 유저가 어떤 영화를 봤는지
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,title,count
0,1,toy story (1995),5
19773,1,pocahontas (1995),5
35123,1,apollo 13 (1995),5
54895,1,star wars: episode iv - a new hope (1977),4
116019,1,schindler's list (1993),5
119008,1,"secret garden, the (1993)",4
130190,1,aladdin (1992),4
138999,1,snow white and the seven dwarfs (1937),4
139706,1,beauty and the beast (1991),5
142605,1,fargo (1996),4


## 2. 분석하기
ratings에 있는 유니크한 영화 개수


In [136]:
data['title'].nunique()

3628

rating에 있는 유니크한 사용자 수

In [137]:
data['user_id'].nunique()

6039

가장 인기 있는 영화 30개(인기순)

In [138]:
movie_count = data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

유저별 평가한 영화 수 통계

In [139]:
user_count = data.groupby('user_id')['count'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: count, dtype: float64

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [140]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Jumanji (1995)' ,'Star Wars: Episode IV - A New Hope (1977)' ,'Matrix, The (1999)','terminator 2: judgment day (1991)','Toy Story (1995)']

# '123'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
# my_movielist = pd.DataFrame({'user_id': ['123']*5, 'movie_id': data[data["title"].isin(my_favorite)]["movie_id"], 'title': my_favorite, 'count': [5]*5})
my_movielist = pd.DataFrame({'user_id': ['123']*5, 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':['123']})['user_id'].any():  # user_id에 '123'이라는 데이터가 없다면
    data = data.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,count
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3
836473,5682,"contender, the (2000)",3
836474,5812,"contender, the (2000)",4
836475,5831,"contender, the (2000)",3
836476,5837,"contender, the (2000)",4
836477,5998,"contender, the (2000)",4


In [141]:
#user_id와 title에 인덱싱을 하는 작업
# 고유한 유저, title를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, title indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [142]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jungin']) # 6039-1 ????
print(artist_to_idx['Toy Story (1995)']) #3628+5(전체영화수)-1(인덱스는0부터) = 3632

KeyError: ignored

In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수 : dic.get('key') = value
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

## 4. CSR matrix 만들기

In [143]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.count, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

ValueError: ignored

In [106]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606590 sha256=e5623be69a95eb116c6922d68d90015b3adccba8d3a8b9b223acb400370e3fd1
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [107]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시키기

AlternatingLeastSquares 클래스의 __init__ 파라미터

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [108]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [109]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

NameError: ignored

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 6.내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


- jungin 벡터와 toystory의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [111]:
jungin, toystory = user_to_idx['jungin'], artist_to_idx['toystory']
jungin_vector, toystory_vector = als_model.user_factors[jungin], als_model.item_factors[toystory]

TypeError: ignored

In [ ]:
jungin_vector

In [ ]:
toystory_vector

In [ ]:
# jungin과 toystory를 내적하는 코드
np.dot(jungin_vector, toystory_vector)

In [ ]:
#랜덤 영화에 대한 선호도를 어떻게 예측할지?
queen = title_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(jungin_vector, queen_vector)

## 7.내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.



In [ ]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

In [ ]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

In [ ]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

In [ ]:
get_similar_artist('2pac')

In [ ]:
get_similar_artist('lady gaga')

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [ ]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

In [ ]:
[idx_to_artist[i[0]] for i in artist_recommended]

In [ ]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [ ]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

## 9. 회고